## Clean buoy data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
buoypath = "../data/data//plank_ChesapeakeBay_all_buoys_clean.csv"
data = pd.read_csv(buoypath)

In [5]:
def get_valid_value(row, col1, col2, min_valid, max_valid):
    
    if pd.notna(row[col1]) and min_valid <= row[col1] <= max_valid:
        return row[col1]
    elif pd.notna(row[col2]) and min_valid <= row[col2] <= max_valid:
        return row[col2]
    else:
        return np.nan


def merge_valid_if_equal(df, col1, col2, constraint_type, newcol=None, show_diffs=True):
    # Check if the columns are the same
    condition =df[col1] != df[col2]
    df_diffs = df[condition]
    df.loc[condition, col1] = df.loc[condition, col1].fillna(df[col1])
    df.loc[condition, col2] = df.loc[condition, col2].fillna(df[col2])
    # df = df.fillna(-999)
    if show_diffs:
        print(df_diffs[[col1, col2]])
    if newcol is None:
        newcol = constraint_type
    if constraint_type == "Latitude":
        minval, maxval = -90, 90
    if constraint_type == "Longitude":
        minval, maxval = -180, 180
    # if df[col1].equals(df[col2]):
    # Merge the columns into a new column
    df[newcol] = df.apply(get_valid_value, axis=1, args=(col1, col2, minval, maxval))
    df.drop([col1, col2], axis=1, inplace=True)
    return df
    # else:
    #     raise ValueError("The columns are not identical, cannot merge")


# def add_constraints(df, col, constraint_type):
#     if constraint_type == "Latitude":
#         df_filtered = df[(df[col] >= -90) & (df[col] <= 90)]

#     if constraint_type == "Longitude":
#         df_filtered = df[(df[col] >= -180) & (df[col] <= 180)]

#     return df_filtered

In [6]:
data_temp1 = merge_valid_if_equal(data, 'Latitude_x','Latitude_y', 'Latitude')
data_temp2 = merge_valid_if_equal(data_temp1, 'Longitude_x','Longitude_y', 'Longitude')

         Latitude_x  Latitude_y
0          39.20141         NaN
1          39.20141         NaN
2          39.20141         NaN
3           0.00000         NaN
4          39.20142         NaN
...             ...         ...
6222417         NaN         NaN
6222418         NaN         NaN
6222419         NaN         NaN
6222420         NaN         NaN
6222421         NaN         NaN

[3700815 rows x 2 columns]
         Longitude_x  Longitude_y
0          -76.57479          NaN
1          -76.57486          NaN
2          -76.57480          NaN
3                NaN          NaN
4          -76.57483          NaN
...              ...          ...
6222417          NaN          NaN
6222418          NaN          NaN
6222419          NaN          NaN
6222420          NaN          NaN
6222421          NaN          NaN

[3719365 rows x 2 columns]


In [7]:
data_clean = data_temp2.replace([np.inf, -np.inf], np.nan)
missing_percentage = data_clean.isnull().mean()
# print(missing_percentage)
clean_columns = data_clean.columns.tolist()

# Drop columns where more than 90% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
data_clean = data.dropna(subset=['Chlorophyll'])

Index([], dtype='object')


In [8]:
del data
del data_temp1
del data_temp2

In [9]:
#remove outliers
pd.set_option('display.max_columns', None)

# data_clean.min()
# data_clean.max()

# humidity is in percentage and the maximum are in reasonable range, take min(current value, 100)
data_clean['Humidity'] = data_clean['Humidity'].apply(lambda x: min(x, 100.0))

In [10]:
# -50< Air temperature <50
# Air pressure is around 1000 milibars
# Humidity is in percentage
# 0 <= Wind speed < 150
# -50 < Temperature < 50
# 0 < Salinity < 50 (usually 25 is max)
# 0 < Chlorophyll < 500 (usually 100 is max)
# 0 < nephelometric turbidity units
# 0 < oxygen < 20

data_clean = data_clean.drop(data_clean.index[
    (data_clean['Air Temperature']<-50) |
    (data_clean['Air pressure'] < 800) |
    (data_clean['Humidity'] < 0) |
    (data_clean['Wind speed'] < 0) |
    (data_clean['Temperature'] < -50) |
    (data_clean['Salinity'] < 0) |
    (data_clean['Chlorophyll'] < 0) |
    (data_clean['Turbidity'] < 0) |
    (data_clean['Oxygen'] < 0) |
    (data_clean['Air Temperature'] > 50) |
    (data_clean['Air pressure'] > 2000) |
    (data_clean['Wind speed'] > 150) |
    (data_clean['Temperature'] > 50) |
    (data_clean['Salinity'] > 50) |
    (data_clean['Chlorophyll'] > 500) |
    (data_clean['Oxygen'] > 20)
    ].tolist())

# surface currents are not accurate
# data_clean = data_clean.drop(columns= ['North surface currents', 'East surface currents'])

In [11]:
data_clean.columns

Index(['Air Temperature QC', 'Air Temperature', 'Air pressure QC',
       'Air pressure', 'Humidity QC', 'Humidity', 'Wind speed QC',
       'Wind speed', 'Wind Direction QC', 'Wind Direction', 'Temperature QC',
       'Temperature', 'Salinity QC', 'Salinity', 'Chlorophyll QC',
       'Chlorophyll', 'Turbidity QC', 'Turbidity', 'Oxygen QC', 'Oxygen',
       'Waves QC', 'Significant wave height', 'Wave from direction',
       'Wave period', 'North surface currents', 'East surface currents',
       'Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour',
       'Sample_minute', 'Sample_second', 'Latitude', 'Longitude'],
      dtype='object')

In [12]:
# delete QC columns
data_clean = data_clean.drop(columns= ['Air Temperature QC','Air pressure QC','Humidity QC','Wind speed QC','Wind Direction QC', 'Temperature QC','Salinity QC','Chlorophyll QC','Turbidity QC','Oxygen QC','Waves QC'])

In [13]:
data_sorted_buoy = data_clean.sort_values(by=['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour','Sample_minute'])
# data_sorted_buoy = data_sorted_buoy.reset_index()
del data_clean

data_sorted_buoy.to_csv('../data/to_merge_buoy.csv', index=False)

del data_sorted_buoy

## Clean water quality data

In [14]:
waterqualitypath = "../data/plankton-patrol_ChesapeakeWaterQuality.csv"
data = pd.read_csv(waterqualitypath)

FileNotFoundError: [Errno 2] No such file or directory: '../data/plankton-patrol_ChesapeakeWaterQuality.csv'

In [ ]:
data["Qualifier"] = data["Qualifier"].replace(np.nan, "=")


columns_to_exclude = ["Parameter", "MeasureValue", "Unit"]
unique_columns = [col for col in data.columns if col not in columns_to_exclude]

df_unique = data[unique_columns].drop_duplicates(subset="EventId")
print(df_unique.shape, data.shape)
data_r = data.pivot_table(
    index=["EventId"], columns="Parameter", values="MeasureValue", aggfunc="first"
).reset_index()
exclude_from_pivoted = ["Parameter", "MeasureValue", "Unit", "SampleDate", "SampleTime"]
pivoted_columns = data_r.columns.tolist()
for ce in exclude_from_pivoted:
    if ce in pivoted_columns:
        pivoted_columns.remove(ce)
data_m = pd.merge(df_unique, data_r, on="EventId", how="left")


print(data_m.columns, data_m.shape)

In [ ]:
data_m.head()

In [ ]:
columns_to_drop = ['EventId','CBSeg2003','Cruise','Program','Project','Agency','Source','Lab','TierLevel']

# Drop the specified columns
data_md = data_m.drop(columns=columns_to_drop)


In [ ]:
set_numeric = [
    "TotalDepth",
    "UpperPycnocline",
    "LowerPycnocline",
    "Depth",
    "Latitude",
    "Longitude",
    "MeasureValue",
    "CHLA",
    "DIN",
    "DO",
    "DOC",
    "DON",
    "DOP",
    "DO_SAT_P",
    "FSS",
    "KD",
    "NH4F",
    "NO23F",
    "NO2F",
    "NO3F",
    "PC",
    "PH",
    "PHEO",
    "PIP",
    "PN",
    "PO4F",
    "PP",
    "Parameter",
    "SALINITY",
    "SECCHI",
    "SIF",
    "SIGMA_T",
    "SPCOND",
    "TDN",
    "TDP",
    "TN",
    "TON",
    "TP",
    "TSS",
    "TURB_NTU",
    "VSS",
    "WTEMP",
]
# set_string = ['
set_date = ["SampleDate", "SampleTime"]


def combine_date_time_strings(df, date_col, time_col):
    # Combine date and time strings
    combined_col = df[date_col] + " " + df[time_col]
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col] = pd.to_datetime(combined_col, errors="coerce")
    df[datetime_col + "_year"] = df[datetime_col].dt.year
    df[datetime_col + "_month"] = df[datetime_col].dt.month
    df[datetime_col + "_day"] = df[datetime_col].dt.day
    df[datetime_col + "_hour"] = df[datetime_col].dt.hour
    df[datetime_col + "_minute"] = df[datetime_col].dt.minute
    df[datetime_col + "_second"] = df[datetime_col].dt.second
    newcolnames = [
        f"{datetime_col}_year",
        f"{datetime_col}_month",
        f"{datetime_col}_day",
        f"{datetime_col}_hour",
        f"{datetime_col}_minute",
        f"{datetime_col}_second",
    ]
    # Drop the original date, time, and combined datetime columns
    df.drop(columns=[date_col, time_col, datetime_col], inplace=True)

    return df, newcolnames


# Function to convert columns to appropriate types
def convert_dtypes(df):
    for col in df.columns:
        df[col] = df[col].replace("nan", np.nan)
        # print(df[col][564463])
        print(f"converting column {col}", end="\t")
        print(df[col].dtype)

        if col not in (set_numeric + set_date):
            df[col] = pd.Categorical(df[col])
            print("Categorical")
        elif col in set_numeric:
            # try:
            # Try converting to numeric (float)
            df[col] = pd.to_numeric(df[col], errors="coerce")
            print("Numeric")
        # except (ValueError, TypeError):
        # try:
        elif col in set_date:
            print("Date, skipped")
            pass
            # Try converting to datetime (date)
            # df[col] = pd.to_datetime(df[col], errors='coerce')
            # except (ValueError, TypeError):
            # Check for categorical
            # unique_ratio = df[col].nunique() / df[col].count()
            # if unique_ratio < 0.2:  # heuristic for categorical, adjustable threshold
            # Convert to string if not numeric, date, or categorical
        else:
            print(f"{col}: string")
            df[col] = df[col].astype(str)
            print("string")
        print(df[col].dtype)
    return df


# Apply the conversion function
datacopy = data_md.__deepcopy__()
data_conv = convert_dtypes(datacopy)
data_cleana, newcols = combine_date_time_strings(data_conv, "SampleDate", "SampleTime")
# Check the result
print(data_cleana.dtypes)
print(data_cleana.shape)
# data.dropna(axis='TotalDepth',how='all')
data_cleana = data_cleana[:-1]
print(data_cleana.shape)

# data_conv = convert_dtypes(data_conv)
# for col in data.columns:
#     # print(data[col])
#     if col not in (set_float + set_date):
#         data[col]= pd.Categorical(df[col])(data[col][1:])
#     # if col in set_string:
#     #     data[col][1:]=data[col][1:].astype("string")
#     if col in set_float:
#         data[col]= pd.to_numeric(data[col], errors='raise')
#     if col in set_date:
#         print(col)
#         data[col][1:]=pd.to_datetime(data[col][1:],errors='coerce')
#         # data[col]=data[col].dt.strftime(f'%m/%d/%Y')
data_cleana

In [ ]:
pivoted_columns= pivoted_columns+newcols
print(len(pivoted_columns), pivoted_columns)

In [ ]:
# pivoted_columns = [col for col in pivoted_columns if col in data_clean.columns]

# data_clean = data_clean.replace([np.inf, -np.inf], np.nan).dropna(
#     subset=[pivoted_columns]
# )
data_cleana = data_cleana.replace([np.inf, -np.inf], np.nan)


missing_percentage = data_cleana.isnull().mean()
print(missing_percentage)
clean_columns = data_cleana.columns.tolist()

# Drop columns where more than 95% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
data_cleana = data_cleana.drop(columns=columns_to_drop)
data_clean = data_cleana.copy()
data_clean = data_cleana.dropna(subset=['CHLA'])
# data_clean = data_cleana.dropna(
#     subset=[
#         "CHLA",
#         "DIN",
#         "DO",
#         "DOC",
#         "DON",
#         "DOP",
#         "DO_SAT_P",
#         "FSS",
#         "KD",
#         "NH4F",
#         "NO23F",
#         "NO2F",
#         "NO3F",
#         "PC",
#         "PH",
#         "PHEO",
#         "PN",
#         "PO4F",
#         "PP",
#         "SALINITY",
#         "SECCHI",
#         "SIF",
#         "SIGMA_T",
#         "SPCOND",
#         "TDN",
#         "TDP",
#         "TN",
#         "TON",
#         "TP",
#         "TSS",
#         "VSS",
#         "WTEMP",
#     ]
# )
dropped_columns = [col for col in clean_columns if col not in data_clean.columns]

data_clean.info()

In [ ]:
print(dropped_columns)

In [ ]:
data_clean.columns

In [ ]:
columns_to_convert = ['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour', 'Sample_minute', 'Sample_second']
data_clean[columns_to_convert] = data_clean[columns_to_convert].astype(int)

In [ ]:
data_clean.drop(columns=['Station','Layer', 'SampleType', 'SampleReplicateType','Qualifier','Method'])

In [ ]:
data_sorted_water = data_clean.sort_values(by=['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour','Sample_minute','Sample_second'])

In [ ]:
data_sorted_water

In [ ]:
data_sorted_water.to_csv('../data/to_merge_water.csv', index=False)

## Toxic data (deprecated)

In [ ]:
combinedbuoyspath = "../data/ToxicsChemicalContaminantHUC8_updated.csv"
# New file with deleted row no 131050, for 'new' file row no 275590

# data = pd.read_csv(combinedbuoyspath, on_bad_lines='skip')
data = pd.read_csv(combinedbuoyspath,sep=',')
# data.info()

In [ ]:
data['SampleDateTime']

In [ ]:
data_r = data.pivot_table(
    index=["SampleDateTime"], columns="ChemicalName", values="ReportedValue", aggfunc="first"
).reset_index()

In [ ]:
pivotcols = data_r.columns
print(pivotcols.shape)
data_r

In [ ]:
def combine_date_time_strings(df, originaldtcolumn):
    # Combine date and time strings
    combined_col = df[originaldtcolumn]
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col] = pd.to_datetime(combined_col, errors="coerce")
    df[datetime_col + "_year"] = df[datetime_col].dt.year
    df[datetime_col + "_month"] = df[datetime_col].dt.month
    df[datetime_col + "_day"] = df[datetime_col].dt.day
    df[datetime_col + "_hour"] = df[datetime_col].dt.hour
    df[datetime_col + "_minute"] = df[datetime_col].dt.minute
    df[datetime_col + "_second"] = df[datetime_col].dt.second
    newcolnames = [
        f"{datetime_col}_year",
        f"{datetime_col}_month",
        f"{datetime_col}_day",
        f"{datetime_col}_hour",
        f"{datetime_col}_minute",
        f"{datetime_col}_second",
    ]
    # Drop the original date, time, and combined datetime columns
    df.drop(columns=[originaldtcolumn, datetime_col], inplace=True)

    return df, newcolnames

In [ ]:
columns_to_exclude = ["ChemicalName", "ReportedValue", "Unit", "Qualifier"]
unique_columns = [col for col in data.columns if col not in columns_to_exclude]
df_unique = data[unique_columns].drop_duplicates(subset="SampleDateTime")
exclude_from_pivoted = ["ChemicalName", "ReportedValue", "Unit", "Qualifier"]
data_cleana, newcols = combine_date_time_strings(data_r, "SampleDateTime")
newcols


In [ ]:
data_cleana = data_cleana.replace([np.inf, -np.inf], np.nan)
missing_percentage = data_cleana.isnull().mean()
# print(missing_percentage)
clean_columns = data_cleana.columns.tolist()

# Drop columns where more than 95% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 0.90].index
print(columns_to_drop)
# data_clean = data_clean.dropna(subset=['Chlorophyll'])

In [ ]:
data_sorted_toxic = data_cleana.sort_values(by=['Sample_year', 'Sample_month', 'Sample_day', 'Sample_hour','Sample_minute','Sample_second'])

In [ ]:
data_sorted_toxic

## Merge buoy and water quality data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
buoypath = "../data/to_merge_buoy.csv"
data_buoy = pd.read_csv(buoypath)

waterpath = "../data/to_merge_water.csv"
data_water = pd.read_csv(waterpath)

In [ ]:
data_buoy['Sample_datetime'] = pd.to_datetime(dict(year=data_buoy.Sample_year,month= data_buoy.Sample_month,day=data_buoy.Sample_day,hour=data_buoy.Sample_hour,minute=data_buoy.Sample_minute,second=data_buoy.Sample_second))

data_water['Sample_datetime'] = pd.to_datetime(dict(year=data_water.Sample_year,month= data_water.Sample_month,day=data_water.Sample_day,hour=data_water.Sample_hour,minute=data_water.Sample_minute,second=data_water.Sample_second))

# data_buoy.set_index('Sample_datetime',inplace=True)
# data_buoy_hourly = data_buoy.resample('H').mean()

# data_water.set_index('Sample_datetime',inplace=True)
# data_water_hourly = data_water.resample('H').mean()

In [ ]:
data_merge = pd.merge(data_buoy,data_water, on = ['Sample_datetime','Latitude','Longitude'], how = 'outer')

In [ ]:
data_merge = data_merge.drop(columns = ['index','Sample_year_x', 'Sample_month_x','Sample_day_x','Sample_hour_x','Sample_minute_x','Sample_second_x','Sample_year_y', 'Sample_month_y','Sample_day_y','Sample_hour_y','Sample_minute_y','Sample_second_y'])

# helper functions
def combine_date_time_strings(df):
    # Convert the combined string to datetime
    datetime_col = "Sample"

    df[datetime_col + "_year"] = df['Sample_datetime'].dt.year
    df[datetime_col + "_month"] = df['Sample_datetime'].dt.month
    df[datetime_col + "_day"] = df['Sample_datetime'].dt.day
    df[datetime_col + "_hour"] = df['Sample_datetime'].dt.hour
    df[datetime_col + "_minute"] = df['Sample_datetime'].dt.minute
    df[datetime_col + "_second"] = df['Sample_datetime'].dt.second
    return df

data_merge = combine_date_time_strings(data_merge)

In [ ]:
data_merge.to_csv('../data/plank_Chesapeake_buoywater_merged.csv', index=False)

## Split merged data into seasons

In [ ]:
# winter : Dec - Feb
# spring : Mar - May
# summer : Jun - Aug
# fall   : Sep - Nov

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'fall'

data_merge['season'] = data_merge['Sample_month'].apply(get_season)


In [ ]:
winter = data_merge[data_merge['season'] == 'winter']
spring = data_merge[data_merge['season'] == 'spring']
summer = data_merge[data_merge['season'] == 'summer']
fall = data_merge[data_merge['season'] == 'fall']

winter.to_csv('../data/plank_Chesapeake_buoywater_merged_winter.csv', index=False)
spring.to_csv('../data/plank_Chesapeake_buoywater_merged_spring.csv', index=False)
summer.to_csv('../data/plank_Chesapeake_buoywater_merged_summer.csv', index=False)
fall.to_csv('../data/plank_Chesapeake_buoywater_merged_fall.csv', index=False)

## XGBoost and SHAP for seasons

In [ ]:
import shap
import xgboost
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
# import cupy as cp

In [ ]:
seasons = ['winter','spring','summer','fall']
for season in seasons:
    print("------------------------")
    print(season)
    print("------------------------")
    
    filepath = "../data/plank_Chesapeake_buoywater_merged_" + season +".csv"
    data = pd.read_csv(filepath)
    
    # drop columns with dtype = object
    # drop surface currents since the values are too abnormal
    object_columns = data.select_dtypes(include=['object']).columns
    data = data.drop(columns=object_columns)
    data = data.drop(columns = ['North surface currents','East surface currents'])
    
    X, y = data.drop(['Chlorophyll'], axis=1), data['Chlorophyll']
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    df = X_train
    print("Original DataFrame:")
    print(df)

    # Identify NaNs in the DataFrame
    nan_locations = df.isna()
    print("\nLocations of NaNs in the DataFrame:")
    print(nan_locations)

    # Count NaNs in each column
    nan_count_per_column = df.isna().sum()
    print("\nCount of NaNs in each column:")
    print(nan_count_per_column)

    # Count NaNs in each row
    nan_count_per_row = df.isna().sum(axis=1)
    print("\nCount of NaNs in each row:")
    print(nan_count_per_row)

    # Rows with at least one NaN
    rows_with_nans = df[df.isna().any(axis=1)]
    print("\nRows with at least one NaN:")
    print(rows_with_nans)

    # Columns with at least one NaN
    columns_with_nans = df.columns[df.isna().any()].tolist()
    print("\nColumns with at least one NaN:")
    print(columns_with_nans)
    
    # params = dict()
    # params["device"] = "cuda"
    # params["tree_method"] = "hist"

    model = xgboost.XGBRegressor(missing=np.nan, enable_categorical=True, device="cuda")
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print("MSE: %f" % (mse))
    
    explainer = shap.Explainer(model=model, masker=X_train)
    explainer.__class__

    # explainer = shap.TreeExplainer(model, data=cp.asarray(X_train_cp))
    # shap_values = explainer.shap_values(cp.asarray(X_test_cp))

    # Convert Shapley values back to NumPy arrays for compatibility
    # shap_values = cp.asnumpy(shap_values)
    
    shap_values = explainer(X_test)
    
    print(shap.plots.waterfall(shap_values[0], max_display=58))
    print(shap.plots.bar(shap_values, max_display=58))
    print(shap.plots.beeswarm(shap_values,max_display=58))